# Coursera Capstone Project - Week 4

## Introduction: Business Problem <a name="introduction"></a>

New York City is most densely populated major city in the United States. With an estimated population of 8.3 million in 2019. It is composed of five boroughs, Brooklyn, Queens, Manhattan, the Bronx, and Staten Island. In this project we will find an optimal location for stakeholders who are intrested in opening a Filipino Restaurant in one of the Boroughs of New York City. This project will take into account several aspects including restaurant density and diversity within the area. It will also include proximity to city centers and amount of foot traffic within the vicinity. 

This project will incorporate several data science techniques to find the optimal location based on several criterias. It will also list several other choices that each of which will have their advantages and disadvantages compared to the best possible location.

## Data <a name="data"></a>

Selection Criteria for our Business Problem:
* Restaurant Density within an area
* Restaurant Diversity within an area
* Distance from city center
* Foot traffic

Data sources that will be needed for the project:
* Neighbourhoods data from wikipidea or other webpages by web scraping using **BeautifulSoup**
* Latitude and longitude of neighbourhoods using **Google Maps Geodcoding API**
* Number and type of restaurant in an area using **Foursquare API**
